In [12]:
%pip install spacy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from matplotlib import pyplot as plt
import spacy
%matplotlib inline
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [14]:
nlp = spacy.load('ru_core_news_sm')

In [15]:
df = pd.read_csv('banks.csv', sep='\t')

In [16]:
df.head(5)

,idx,Score,Text
0,0,Positive,В Альфа-Банке работает замечательная девушка -...
1,1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,2,Positive,Очень порадовала оперативность работы в банке....
3,3,Negative,Имела неосторожность оформить потреб. кредит в...
4,4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...


In [17]:
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Score'], test_size=0.5, random_state=35)

In [18]:
class Tokenizer:
    def __init__(self):
        self.nlp = spacy.load('ru_core_news_sm')
    
    def tokenize_text(self, text):
        tokens = []
        for sentence in self.nlp(text).sents:
            for token in sentence:
                if not token.is_stop and not token.is_punct:
                    tokens.append(token.text)
        return tokens

tokenizer = Tokenizer()
cData = pd.DataFrame({'text': X_train, 'tokenizer_text': X_train.apply(tokenizer.tokenize_text)})
cData

,text,tokenizer_text
10309,Хочу поблагодарить сотрудниц и руководителя от...,"[поблагодарить, сотрудниц, руководителя, отдел..."
7642,"Всем, кто хочет иметь дело с этим банком, данн...","[хочет, дело, банком, отзыв, обязателен, прочт..."
8951,Плюсы:- Приняли решение за один рабочий день.-...,"[Плюсы:-, Приняли, решение, рабочий, день.-, М..."
2688,Огромное спасибо допофису Ковровский. Наше пре...,"[Огромное, спасибо, допофису, Ковровский, пред..."
4917,"Здравствуйте.Сегодня, 2 марта 2016г в 17:00 на...","[Здравствуйте, Сегодня, 2, марта, 2016, 17:00,..."
...,...,...
3007,Начиталась я тут отрицательных отзывов про Сит...,"[Начиталась, отрицательных, отзывов, Ситибанк,..."
7148,Несколько лет назад мы с дочерью делали ремонт...,"[лет, дочерью, делали, ремонт, квартире, решил..."
9143,Сторонний человек взял кредит в вашем банке и ...,"[Сторонний, человек, взял, кредит, вашем, банк..."
1295,"Ну что... как всегда банкомат зажевал деньги, ...","[банкомат, зажевал, деньги, электронная, прете..."


In [19]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Score'] = df['Score'].apply(lambda x: 1 if x == 'Positive' else 0)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#создание экземпляра CountVectorizer с пользовательским токенизатором
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize_text)
#преобразование обучающих данных в векторизованное представление с использованием CountVectorizer
X_train_vectorized = vectorizer.fit_transform(X_train)
#преобразование тестовых данных в векторизованное представление с использованием обученного CountVectorizer
X_test_vectorized = vectorizer.transform(X_test)

#создание модели путем объединения CountVectorizer и LogisticRegression в пайплайн
logistic_model = make_pipeline(LogisticRegression())
#обучение модели на векторизованных обучающих данных
logistic_model.fit(X_train_vectorized, y_train)

#получение прогнозов на тестовых данных с использованием обученной модели логистической регрессии
y_pred_logistic = logistic_model.predict(X_test_vectorized)

accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print('Accuracy (Logistic Regression):', accuracy_logistic)  # Вывод точности
#вывод отчета о классификации модели логистической регрессии
print('\nClassification Report (Logistic Regression):\n', classification_report(y_test, y_pred_logistic))


c:\Users\misha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\misha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy (Logistic Regression): 0.9438571428571428

Classification Report (Logistic Regression):
               precision    recall  f1-score   support

    Negative       0.93      0.96      0.94      3477
    Positive       0.96      0.93      0.94      3523

    accuracy                           0.94      7000
   macro avg       0.94      0.94      0.94      7000
weighted avg       0.94      0.94      0.94      7000



In [22]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


нс

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Score'], test_size=0.5, random_state=35)

In [27]:
tokenizer = Tokenizer()
#подгонка Tokenizer под текстовые данные в X_train для построения индекса слов на основе частоты слов
tokenizer.fit_on_texts(X_train)
#токенизация текстовых данных в X_train в последовательности токенов с использованием подготовленного Tokenizer
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
#токенизация текстовых данных в X_test в последовательности токенов с использованием того же подготовленного Tokenizer из X_train
X_test_tokenized = tokenizer.texts_to_sequences(X_test)

In [28]:
#применение заполнения (padding) к токенизированным данным X_train_tokenized для выравнивания их длины
X_train_padded = pad_sequences(X_train_tokenized)
#применение заполнения (padding) к токенизированным данным X_test_tokenized с учетом максимальной длины, равной максимальной длине в X_train_padded
X_test_padded = pad_sequences(X_test_tokenized, maxlen=X_train_padded.shape[1])


In [30]:
encoder = LabelEncoder()
# Преобразование категориальных обучающих меток y_train в числовой формат с помощью метода fit_transform
y_train_encoded = encoder.fit_transform(y_train)
# Преобразование категориальных тестовых меток y_test в числовой формат с использованием метода transform, примененного к тем же параметрам, что и в случае y_train
y_test_encoded = encoder.transform(y_test)

In [32]:
model = tf.keras.Sequential([
    #для преобразования входных данных в векторные представления
    tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 100, input_length=X_train_padded.shape[1]),
    #для обработки последовательных данных с использованием 1D сверток
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    #для извлечения наиболее важных признаков из результатов свертки
    tf.keras.layers.GlobalMaxPooling1D(),
    #для бинарной классификации
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#компиляция модели с функцией потерь binary_crossentropy, оптимизатором adam и метрикой accuracy
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [34]:
#обучение модели
model.fit(X_train_padded, y_train_encoded, epochs=10, batch_size=32)

Epoch 1/10


219/219 [==============================] - 182s 820ms/step - loss: 0.4197 - accuracy: 0.7998
Epoch 2/10
219/219 [==============================] - 159s 725ms/step - loss: 0.1433 - accuracy: 0.9487
Epoch 3/10
219/219 [==============================] - 141s 642ms/step - loss: 0.0392 - accuracy: 0.9906
Epoch 4/10
219/219 [==============================] - 155s 710ms/step - loss: 0.0075 - accuracy: 0.9997
Epoch 5/10
219/219 [==============================] - 180s 819ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 6/10
219/219 [==============================] - 168s 766ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 7/10
219/219 [==============================] - 180s 820ms/step - loss: 7.4102e-04 - accuracy: 1.0000
Epoch 8/10
219/219 [==============================] - 169s 771ms/step - loss: 5.1010e-04 - accuracy: 1.0000
Epoch 9/10
219/219 [==============================] - 154s 701ms/step - loss: 3.7060e-04 - accuracy: 1.0000
Epoch 10/10
219/219 [=============================

In [35]:
_, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print('Accuracy: %.1f' % (accuracy * 100))

219/219 [==============================] - 21s 95ms/step - loss: 0.2529 - accuracy: 0.9314
Accuracy: 93.1


трансформер